# 外挿法によるError Mitigationの基礎

このノートブックでは、Error Mitigationの代表的な手法である外挿法(ZNE : Zero Noise Extrapolation)について学ぶ。

1量子ビットに対し、回転ゲートを作用させるだけの簡単な回路を題材に、Error Mitigationの流れを確認する。

In [ ]:
%pip install qiskit==0.45.2 qiskit-aer==0.12.0
%pip install pylatexenc

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Kraus, SuperOp
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from matplotlib import pyplot as plt

from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)

%matplotlib inline

# エラーモデルの定義

In [ ]:
def make_noise_model(p_error: float) -> NoiseModel:
    # p_error : ゲートエラー確率

    bit_flip = pauli_error([('X', p_error), ('I', 1 - p_error)])
    phase_flip = pauli_error([('Z', p_error), ('I', 1 - p_error)])
    bitphase_flip = bit_flip.compose(phase_flip)

    error_gate1 = bitphase_flip
    error_gate2 = error_gate1.tensor(bitphase_flip)

    noise_model = NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_gate1, ["x", "rx"])
    noise_model.add_all_qubit_quantum_error(error_gate2, ["cx"])

    print(noise_model)
    
    return noise_model

In [ ]:
p_noise = 0.05
noise_model = make_noise_model(p_noise)

# 回転ゲートのみの回路で実験

まずは、1量子ビットに対して回転ゲートのみを適用する簡単な回路を作成してみる。

In [ ]:
n_shots = 100_000
theta = np.pi / 4

In [ ]:
n_qubits = 1
circ = QuantumCircuit(n_qubits)

circ.rx(theta, 0)
circ.measure_all()
circ.draw("mpl")

## エラーなしの場合

In [ ]:
sim_ideal = AerSimulator()
result_ideal = sim_ideal.run(circ, shots=n_shots).result()
plot_histogram(result_ideal.get_counts(0))

In [ ]:
true_score = result_ideal.get_counts(0)["1"] / n_shots
print(true_score)

エラーなしの場合、15％弱の割合で1が出現する。

## エラーありの場合

この回路に対して、エラー率が5％のノイズモデルを動かしてみる。

In [ ]:
sim_noise = AerSimulator(noise_model=noise_model)
result_noise = sim_noise.run(circ, shots=n_shots).result()
counts_noise = result_noise.get_counts(0)

# Plot noisy output
plot_histogram(counts_noise)

In [ ]:
noise_score = result_noise.get_counts(0)["1"] / n_shots
print(noise_score)

エラーが増えることで、1の出現頻度が増え、18％程度になっている。

# エラーを増幅する

ここで、エラーの発生確率を2倍にしたモデルを作成してみる。

In [ ]:
noise_model_2 = make_noise_model(p_noise*2)

In [ ]:
sim_noise_2 = AerSimulator(noise_model=noise_model_2)
result_noise_2 = sim_noise_2.run(circ, shots=n_shots).result()
counts_noise_2 = result_noise_2.get_counts(0)

# Plot noisy output
plot_histogram(counts_noise_2)

In [ ]:
noise_score_2 = result_noise_2.get_counts(0)["1"] / n_shots
print(noise_score_2)

さらにエラーが増え、22％程度のエラー発生率になっている。

回路中のゲートエラー率が5％増えたことで、出力の1の出現頻度が18％→22％に増えた。

ここで、（エラー率10％の回路の出力 - エラー率5％の回路の出力）がエラーが5％増えることの効果とみなして，

これをエラー率5％の回路の出力から引き算することで，仮想的にエラー率0％の回路の出力を求める。

In [ ]:
mitigated_score = noise_score - (noise_score_2 - noise_score)
print(mitigated_score)

多少差はあるが，何も補正しないよりは良い結果が得られた。

図でイメージを示すと、以下の通りである。

In [ ]:
plt.plot([p_noise, p_noise*2],[noise_score, noise_score_2], "o-")
plt.plot([0., p_noise],[mitigated_score, noise_score], "o--")
plt.scatter([0],[true_score],c="r")
plt.xticks([0.01 * i for i in range(11)]);

# 回路を長くする方法でのエラー増幅

実際の量子コンピュータでは、ゲートエラー率を正確に2倍にするような操作を、一般のユーザが行うことは難しいことが多い。

ここで、同じ出力になるような長い回路を作成することで、エラーを増幅させることを考える。

$ U = U U^{\dagger} U $ とすることで，回路を長くして，エラーを増幅することができる。

これにより、エラーが3倍に増える。

In [ ]:
# System Specification
n_qubits = 1
circ_2 = QuantumCircuit(n_qubits)

# Test Circuit
circ_2.rx(theta, 0)
circ_2.rx(-theta, 0)
circ_2.rx(theta, 0)
circ_2.measure_all()
circ_2.draw("mpl")

In [ ]:
# Transpile circuit for noisy basis gates
result_noise_3 = sim_noise.run(circ_2, shots=n_shots).result()
counts_noise_3 = result_noise_3.get_counts(0)

# Plot noisy output
plot_histogram(counts_noise_3)

In [ ]:
noise_score_3 = result_noise_3.get_counts(0)["1"] / n_shots
print(noise_score_3)

同様の理屈で，エラー率が0％のときの回路の出力を推定する

In [ ]:
mitigated_score_2 = noise_score - (noise_score_3 - noise_score) / 2
print(mitigated_score_2)

In [ ]:
plt.plot([1, 3],[noise_score, noise_score_3], "o-")
plt.plot([0, 1],[mitigated_score_2, noise_score], "o--")
plt.scatter([0],[true_score],c="r")
plt.xticks([1 * i for i in range(4)]);

# さらにエラーを増幅してみる

2点だけで推定するよりも、より多くの点を取って、推定するほうが正しい推定ができそうに思える。

ここでは、さらにエラーを増幅してみる。

$ U = U (U^{\dagger} U)^n $ とすることで，さらに回路長を長くすることができる。

In [ ]:
def make_repeated_circ(n_repeat: int) -> QuantumCircuit:
    # 指定された回数だけ回路長を増やした回路を作成する
    n_qubits = 1
    circ = QuantumCircuit(n_qubits)

    circ.rx(theta, 0)
    for _ in range(n_repeat):
        circ.rx(-theta, 0)
        circ.rx(theta, 0)
    circ.measure_all()
    
    return circ

In [ ]:
def calc_noise_score(circ):
    # 与えられた回路の出力を計算する
    result_noise = sim_noise.run(circ, shots=n_shots).result()
    counts_noise = result_noise.get_counts(0)
    noise_score = result_noise.get_counts(0)["1"] / n_shots
    return noise_score

In [ ]:
circ_length_list = [1]
score_list = [noise_score]
for i in range(1, 10): # 増幅回数を1〜9で計算する
    circ = make_repeated_circ(i)
    repeated_noise_score = calc_noise_score(circ)
    
    circ_length_list.append(1 + i*2)
    score_list.append(repeated_noise_score)

In [ ]:
score_list

In [ ]:
plt.plot(circ_length_list,score_list, "o-")
plt.xticks([1 * i for i in range(20)]);

回路長に応じた出力の変化がプロットできた。

これに、適切な関数を当てはめれば、良い推定ができそうである。

# 線形回帰を行ってみる

あまり筋がよさそうには見えないが、まずは簡単な線型回帰から行ってみる。

ここでは、後ほどより複雑な関数を使うことを見越して、scipyのcurve_fitを用いる。

In [ ]:
from scipy.optimize import curve_fit

# 関数形を定義
def linear_fitting(x, a, b):
    return a*x + b

In [ ]:
# パラメータを推定する
param, _ = curve_fit(linear_fitting, circ_length_list, score_list)

In [ ]:
# 推定した関数に当てはめてみる
x_arr = np.array([0] + circ_length_list)
fitting_result = [linear_fitting(x, param[0], param[1]) for x in x_arr]

In [ ]:
plt.plot(circ_length_list, score_list, "o-")
plt.plot([0]+circ_length_list, fitting_result, "o-")
plt.scatter([0],[true_score],c="r")
plt.xticks([1 * i for i in range(20)]);

In [ ]:
print(fitting_result[0])
print(true_score)

さすがにあまり当てはまりがよくない。

# 演習：適切な関数で近似してみる

プロットされた形状から、適当な関数を考え、近似してみましょう。

In [ ]:
# xを入力（回路長）とし、残りのパラメータを自由な数設定する。（以下は線型回帰の例）
def my_function(x,a,b):
    return a*x + b

In [ ]:
# パラメータを推定する
param, _ = curve_fit(my_function, circ_length_list, score_list) # paramにはxを除くパラメータがリストで格納される

In [ ]:
# 推定した関数に当てはめてみる
x_arr = np.array([0] + circ_length_list)
fitting_result = [my_function(x, param[0], param[1]) for x in x_arr] # パラメータの個数を合わせること

In [ ]:
plt.plot(circ_length_list, score_list, "o-")
plt.plot([0]+circ_length_list, fitting_result, "o-")
plt.scatter([0],[true_score],c="r")
plt.xticks([1 * i for i in range(20)]);